# general_utils.ipynb
# WESmith 08/04/23
## decorators: see  https://www.youtube.com/watch?v=WpF6azYAxYg

In [ ]:
from functools import cache, wraps
import logging, time
import inspect # to be able to read default kwargs in a decorator
# see https://stackoverflow.com/questions/34832573/python-decorator-to-display-passed-and-default-kwargs

In [ ]:
logging.basicConfig(level=logging.INFO)  # this is needed even though logging decorator sets it

In [ ]:
# decorators

def timer(f):
    def wrapper(*args, **kwargs):
        start  = time.time()
        result = f(*args, **kwargs)
        dt     = time.time() - start
        print(f'time_to_run = {dt}')
        return result
    return wrapper

def logging_decorator(func):  # WS mod: to read default kwargs needs 'inspect' module and some code
    argspec          = inspect.getfullargspec(func)
    positional_count = len(argspec.args) - len(argspec.defaults)
    defaults         = dict(zip(argspec.args[positional_count:], argspec.defaults))
    #print(f'argspec.args: {argspec.args}, argspec.defaults: {argspec.defaults}')
    #print(f'defaults dict: {defaults}')
    @wraps(func)
    def wrapper(*args, **kwargs):
        logger = logging.getLogger(func.__name__)
        logger.setLevel(logging.INFO)  # this doesn't seem to work: need an external global setting
        # ie: logging.basicConfig(level=logging.INFO)  # this is needed even though decorator sets it

        used_kwargs = kwargs.copy()
        # update used_kwargs if defaults have been overridden
        used_kwargs.update(zip(argspec.args[positional_count:], args[positional_count:]))

        # Call the original function and capture the return value
        result = func(*args, **kwargs)

        # Log the input arguments
        dd = {k: used_kwargs.get(k, d) for k, d in defaults.items()}
        logger.info(f"Calling {func.__name__} with args: {args[:positional_count]}, kwargs: {dd}")
        
        # Log the return value
        logger.info(f"{func.__name__} returned: {result}")

        return result

    return wrapper

In [ ]:
# example function to time
#@timer  # WS note: the decorator is always on if set here
# can instead create a new function pf_time = timer(prime_factors);
# there are ways to turn decorators on/off, but this requires more code
def prime_factors(n):
    factors = []
    divisor = 2
    
    while n > 1:
        while n % divisor == 0:
            factors.append(divisor)
            n //= divisor
        divisor += 1

    return factors

In [ ]:
@cache # turn off to see difference in times below
# with it on, fibonacci(33) was 10^6 times faster!
def fibonacci(n):
    if not isinstance(n, int) or n < 1:
        raise ValueError(f'{n} is not a positive integer')
    if n == 1 or n == 2:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)
    
def global_fibonacci(n): # define this for timer wrapper to work
    return fibonacci(n)

In [ ]:
prime_factors(2**9 + 1)

In [ ]:
prime_factors(2**25 + 1)

In [ ]:
# get all primes below a number (inefficient but useful demo)
n  = 2**9 + 1
pp = []
for k in range(n):
    dd = prime_factors(k)
    if len(dd) == 1:
        pp.append(dd[0])
print(f'primes below {n}: {pp}')

In [ ]:
# calling decorator explicitly (see comments above definition of prime_factors())
pf_timer = timer(prime_factors)
pf_timer(2**29 + 1)

In [ ]:
for i in range(1,10):
    print(fibonacci(i))

In [ ]:
fib_time = timer(global_fibonacci)
for i in range(30,36):
    nth_term = fib_time(i)
    print(f'Fibonacci({i}) = {nth_term}')

In [ ]:
# Example usage of the decorator
@logging_decorator
def add(a, b, c=5, d='hello'):
    return a + b + c

@logging_decorator
def multiply(a, b, c=5):
    return a * b * c

In [ ]:
result1 = add(3, 5, c=2, d='bye')
print("Result of add function:", result1)

In [ ]:
add(4,9)

In [ ]:
result2 = multiply(2, 4)
print("Result of multiply function:", result2)